<a href="https://colab.research.google.com/github/IMI1998/pyTorch/blob/main/PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets , transforms
from torch.optim.lr_scheduler import StepLR

In [18]:
class Net(nn.Module):
  def __init__(self):
    super(Net , self).__init__()
    self.conv1 = nn.Conv2d(1,32,3,1)
    self.conv2 = nn.Conv2d(32,64,3,1)
    self.dropout = nn.Dropout(0.25)
    self.dropout2 = nn.Dropout(0.5)
    self.fc1 = nn.Linear(9216 , 128)
    self.fc2 = nn.Linear(128,10)


  def forward(self , x):
    x=self.conv1(x)
    x=F.relu(x)
    x=self.conv2(x)
    x=F.relu(x)
    x=F.max_pool2d(x,2)
    x=self.dropout(x)
    x=torch.flatten(x,1)
    x=self.fc1(x)
    x=F.relu(x)
    x = self.dropout2(x)
    x = self.fc2(x)
    output = F.log_softmax(x , dim=1)
    return output

In [22]:
def train(model , device , train_loader , optimizer , epoch):
  model.train()
  for batch_idx , (data , target) in enumerate(train_loader):
    data , target = data.to(device) , target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output , target)
    loss.backward()
    optimizer.step()
    if batch_idx % 10 == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)] \t loss: {:.6f}'.format(
          epoch , batch_idx * len(data) , len(train_loader.dataset),
          100. * batch_idx / len (train_loader) , loss.item()
      ))

In [32]:
def test(model , device , test_loader):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data , target in test_loader:
      data , target = data.to(device) , target.to(device)
      output = model(data)
      test_loss += F.nll_loss(output , target , reduction='sum').item()
      pred = output.argmax(dim=1 , keepdim=True)
      correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(test_loader.dataset)

  print('\n Test set: Average loss: {:.4f} , Accuracy: {}/{} ({:.0f}%)\n' .format(
      test_loss , correct , len(test_loader.dataset), 100. * correct / len(test_loader.dataset)
  ))


In [33]:
use_cuda = torch.cuda.is_available()
use_mps = torch.backends.mps.is_available()

torch.manual_seed(143421)
if use_cuda:
  device = torch.device("cuda")
elif use_mps:
  device = torch.device("mps")
else:
  device = torch.device("cpu")

train_kwargs = {'batch_size' : 64 }
test_kwargs = {'batch_size' : 1000}

if use_cuda:
  cuda_kwargs = {'num_workers' : 1 ,
                 'pin_memory' : True ,
                 'shuffle' : True
                 }

  train_kwargs.update(cuda_kwargs)
  test_kwargs.update(cuda_kwargs)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307 , ) ,(0.3081 , ) )

])

dataset1 =datasets.MNIST('data' , train=True , download=True , transform=transform)
dataset2 = datasets.MNIST('data' , train=False , transform=transform)

train_loader = torch.utils.data.DataLoader(dataset1 , **train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2 , **train_kwargs)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters() , lr=1.0)

scheduler = StepLR(optimizer , step_size=1 , gamma=0.7)
for epoch in range(1,10):
    train(model , device , train_loader , optimizer , epoch)
    test(model , device , test_loader)
    scheduler.step()

torch.save(model.state_dict() , "mnist_cnn.pt")


Train Epoch: 1 [0/60000 (0%)] 	 loss: 2.287570
Train Epoch: 1 [640/60000 (1%)] 	 loss: 1.529613
Train Epoch: 1 [1280/60000 (2%)] 	 loss: 0.667243
Train Epoch: 1 [1920/60000 (3%)] 	 loss: 0.456806
Train Epoch: 1 [2560/60000 (4%)] 	 loss: 0.289104
Train Epoch: 1 [3200/60000 (5%)] 	 loss: 0.230878
Train Epoch: 1 [3840/60000 (6%)] 	 loss: 0.177960
Train Epoch: 1 [4480/60000 (7%)] 	 loss: 0.341935
Train Epoch: 1 [5120/60000 (9%)] 	 loss: 0.429217
Train Epoch: 1 [5760/60000 (10%)] 	 loss: 0.221784
Train Epoch: 1 [6400/60000 (11%)] 	 loss: 0.308059
Train Epoch: 1 [7040/60000 (12%)] 	 loss: 0.196567
Train Epoch: 1 [7680/60000 (13%)] 	 loss: 0.252638
Train Epoch: 1 [8320/60000 (14%)] 	 loss: 0.074465
Train Epoch: 1 [8960/60000 (15%)] 	 loss: 0.204269
Train Epoch: 1 [9600/60000 (16%)] 	 loss: 0.196687
Train Epoch: 1 [10240/60000 (17%)] 	 loss: 0.270885
Train Epoch: 1 [10880/60000 (18%)] 	 loss: 0.179240
Train Epoch: 1 [11520/60000 (19%)] 	 loss: 0.362878
Train Epoch: 1 [12160/60000 (20%)] 	 loss